In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook as tqdm

#set the max columns to none
pd.set_option('display.max_columns', None)

### I will create the interaction matrix from history data and save it in order to quick access

In [2]:
size = 'demo'
type_ = 'validation'
#Import train Data
history_train_path = f'./files/parquet/ebnerd_{size}/{type_}/history.parquet'

history_df = pd.read_parquet(history_train_path)
print(history_df.shape)

behaviors_path = f'./files/parquet/ebnerd_{size}/{type_}/behaviors.parquet'
behaviors_df = pd.read_parquet(behaviors_path)
print(f'Raw {size} validation behaviors df shape:          ',behaviors_df.shape)


(1562, 5)
Raw demo validation behaviors df shape:           (25356, 17)


In [3]:
history_df.head(2)

,user_id,impression_time_fixed,scroll_percentage_fixed,article_id_fixed,read_time_fixed
0,750497,"[2023-05-04T09:42:39.000000, 2023-05-04T09:43:...","[100.0, 65.0, 100.0, 100.0, 100.0, 100.0, 100....","[9749224, 9749156, 9749224, 9748948, 9748980, ...","[49.0, 5.0, 7.0, 151.0, 214.0, 199.0, 22.0, 64..."
1,22779,"[2023-05-04T07:53:42.000000, 2023-05-04T15:59:...","[52.0, 39.0, 62.0, 38.0, 74.0, 19.0, 30.0, 56....","[9749025, 9750090, 9750015, 9750161, 9745750, ...","[4.0, 16.0, 2.0, 9.0, 40.0, 7.0, 9.0, 8.0, 18...."


In [4]:
behaviors_df.head(2)

,impression_id,article_id,impression_time,read_time,scroll_percentage,device_type,article_ids_inview,article_ids_clicked,user_id,is_sso_user,gender,postcode,age,is_subscriber,session_id,next_read_time,next_scroll_percentage
0,144772,NaN,2023-05-30 14:21:34,29.0,NaN,2,"[9788239, 9780702, 9553264, 9787499, 6741781, ...",[9783042],76658,False,NaN,NaN,NaN,False,29,7.0,59.0
1,144777,NaN,2023-05-30 14:22:11,10.0,NaN,2,"[9788521, 9786217, 9553264, 9788361, 9788352, ...",[9788125],76658,False,NaN,NaN,NaN,False,29,58.0,98.0


In [284]:
print(history_df.explode('impression_time_fixed').impression_time_fixed.min())
print(history_df.explode('impression_time_fixed').impression_time_fixed.max())
history_df.head(2)

2023-05-04 07:00:11
2023-05-25 06:59:54


,user_id,impression_time_fixed,scroll_percentage_fixed,article_id_fixed,read_time_fixed
0,750497,"[2023-05-04T09:42:39.000000, 2023-05-04T09:43:...","[100.0, 65.0, 100.0, 100.0, 100.0, 100.0, 100....","[9749224, 9749156, 9749224, 9748948, 9748980, ...","[49.0, 5.0, 7.0, 151.0, 214.0, 199.0, 22.0, 64..."
1,22779,"[2023-05-04T07:53:42.000000, 2023-05-04T15:59:...","[52.0, 39.0, 62.0, 38.0, 74.0, 19.0, 30.0, 56....","[9749025, 9750090, 9750015, 9750161, 9745750, ...","[4.0, 16.0, 2.0, 9.0, 40.0, 7.0, 9.0, 8.0, 18...."


In [285]:
print(behaviors_df.impression_time.min())
print(behaviors_df.impression_time.max())
behaviors_df.head(2)

2023-05-25 07:00:15
2023-06-01 06:59:33


,impression_id,article_id,impression_time,read_time,scroll_percentage,device_type,article_ids_inview,article_ids_clicked,user_id,is_sso_user,gender,postcode,age,is_subscriber,session_id,next_read_time,next_scroll_percentage
0,144772,NaN,2023-05-30 14:21:34,29.0,NaN,2,"[9788239, 9780702, 9553264, 9787499, 6741781, ...",[9783042],76658,False,NaN,NaN,NaN,False,29,7.0,59.0
1,144777,NaN,2023-05-30 14:22:11,10.0,NaN,2,"[9788521, 9786217, 9553264, 9788361, 9788352, ...",[9788125],76658,False,NaN,NaN,NaN,False,29,58.0,98.0


In [286]:
# Preprocess behaviour
behaviors_df = behaviors_df[['article_ids_clicked','user_id','read_time','scroll_percentage']]
behaviors_df = behaviors_df.explode('article_ids_clicked')


In [287]:
behaviors_df.groupby('user_id')['article_ids_clicked'].apply(list).reset_index()


,user_id,article_ids_clicked
0,19181,"[9783019, 9770145, 9782519, 9782899, 9788947, ..."
1,21271,"[9785113, 9339920, 9786222, 9782407, 9784097, ..."
2,21774,[9785835]
3,22779,"[9787261, 9784852, 9780702, 9785111, 9782806, ..."
4,22895,"[9782996, 9790574, 9790475, 9781013, 9780514, ..."
...,...,...
1557,2581162,"[9781998, 9695098, 9782487, 9780467, 9783404, ..."
1558,2583035,"[9782108, 9781906, 9785835, 9780439, 9783057, ..."
1559,2583477,"[9780210, 9781057, 9786204, 9777846, 9787469, ..."
1560,2584367,"[9785434, 9785111, 9772508]"


In [288]:
behaviors_df.groupby('user_id')['read_time'].apply(list).reset_index()


,user_id,read_time
0,19181,"[15.0, 6.0, 5.0, 21.0, 57.0, 18.0, 19.0, 13.0,..."
1,21271,"[23.0, 30.0, 15.0, 16.0, 83.0, 10.0, 25.0]"
2,21774,[47.0]
3,22779,"[14.0, 58.0, 11.0, 14.0, 10.0, 11.0, 9.0, 61.0..."
4,22895,"[14.0, 8.0, 29.0, 17.0, 17.0, 9.0, 8.0, 14.0, ..."
...,...,...
1557,2581162,"[52.0, 20.0, 34.0, 16.0, 104.0, 22.0, 52.0, 83..."
1558,2583035,"[12.0, 20.0, 15.0, 22.0, 156.0, 8.0, 23.0, 8.0..."
1559,2583477,"[76.0, 38.0, 411.0, 15.0, 28.0, 19.0, 22.0, 35.0]"
1560,2584367,"[44.0, 110.0, 51.0]"


In [289]:
behaviors_df_merged= pd.merge(behaviors_df.groupby('user_id')['article_ids_clicked'].apply(list).reset_index(),behaviors_df.groupby('user_id')['read_time'].apply(list).reset_index(),on='user_id')

In [290]:
behaviors_df

,article_ids_clicked,user_id,read_time,scroll_percentage
0,9783042,76658,29.0,NaN
1,9788125,76658,10.0,NaN
2,9782806,760446,16.0,NaN
3,9782656,760446,25.0,NaN
4,9777324,760446,11.0,NaN
...,...,...,...,...
25351,9783317,1167944,63.0,100.0
25352,9754112,1167944,23.0,100.0
25353,9782517,1171223,21.0,100.0
25354,9783004,1358150,10.0,NaN


In [291]:
behaviors_df= pd.merge(behaviors_df_merged,behaviors_df.groupby('user_id')['scroll_percentage'].apply(list).reset_index(),on='user_id')

In [292]:
behaviors_df.head(5)

,user_id,article_ids_clicked,read_time,scroll_percentage
0,19181,"[9783019, 9770145, 9782519, 9782899, 9788947, ...","[15.0, 6.0, 5.0, 21.0, 57.0, 18.0, 19.0, 13.0,...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ..."
1,21271,"[9785113, 9339920, 9786222, 9782407, 9784097, ...","[23.0, 30.0, 15.0, 16.0, 83.0, 10.0, 25.0]","[nan, nan, nan, nan, 100.0, 100.0, nan]"
2,21774,[9785835],[47.0],[nan]
3,22779,"[9787261, 9784852, 9780702, 9785111, 9782806, ...","[14.0, 58.0, 11.0, 14.0, 10.0, 11.0, 9.0, 61.0...","[nan, 100.0, nan, nan, nan, nan, nan, nan, nan]"
4,22895,"[9782996, 9790574, 9790475, 9781013, 9780514, ...","[14.0, 8.0, 29.0, 17.0, 17.0, 9.0, 8.0, 14.0, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ..."


In [293]:
try:
    history_df.drop('impression_time_fixed', axis=1, inplace=True) # i will drop this column initialy even though i can proccess it and use it
except:
    pass

In [294]:
history_df['scroll_percentage_fixed_len'] = history_df['scroll_percentage_fixed'].apply(lambda lst: len(lst))
history_df['article_id_fixed_len'] = history_df['article_id_fixed'].apply(lambda lst: len(lst))
history_df['read_time_fixed_len'] = history_df['read_time_fixed'].apply(lambda lst: len(lst))

In [295]:
history_df.head(2)

,user_id,scroll_percentage_fixed,article_id_fixed,read_time_fixed,scroll_percentage_fixed_len,article_id_fixed_len,read_time_fixed_len
0,750497,"[100.0, 65.0, 100.0, 100.0, 100.0, 100.0, 100....","[9749224, 9749156, 9749224, 9748948, 9748980, ...","[49.0, 5.0, 7.0, 151.0, 214.0, 199.0, 22.0, 64...",409,409,409
1,22779,"[52.0, 39.0, 62.0, 38.0, 74.0, 19.0, 30.0, 56....","[9749025, 9750090, 9750015, 9750161, 9745750, ...","[4.0, 16.0, 2.0, 9.0, 40.0, 7.0, 9.0, 8.0, 18....",393,393,393


### The lenths of the lists in each column seem equal

In [296]:
history_df['scroll_percentage_fixed_has_0_in'] = history_df['scroll_percentage_fixed'].apply(lambda lst: 0 in lst)
history_df['read_time_fixed_has_0_in'] = history_df['read_time_fixed'].apply(lambda lst: 0 in lst)
history_df['read_time_fixed_has_nan'] = history_df['read_time_fixed'].apply(lambda lst: pd.isna(lst.any()))

In [297]:
history_df.head()

,user_id,scroll_percentage_fixed,article_id_fixed,read_time_fixed,scroll_percentage_fixed_len,article_id_fixed_len,read_time_fixed_len,scroll_percentage_fixed_has_0_in,read_time_fixed_has_0_in,read_time_fixed_has_nan
0,750497,"[100.0, 65.0, 100.0, 100.0, 100.0, 100.0, 100....","[9749224, 9749156, 9749224, 9748948, 9748980, ...","[49.0, 5.0, 7.0, 151.0, 214.0, 199.0, 22.0, 64...",409,409,409,False,True,False
1,22779,"[52.0, 39.0, 62.0, 38.0, 74.0, 19.0, 30.0, 56....","[9749025, 9750090, 9750015, 9750161, 9745750, ...","[4.0, 16.0, 2.0, 9.0, 40.0, 7.0, 9.0, 8.0, 18....",393,393,393,False,True,False
2,373598,"[nan, nan, nan, 59.0, 33.0, 75.0, nan, nan, 76...","[9514481, 9514481, 9111040, 9750389, 9750307, ...","[0.0, 0.0, 0.0, 3.0, 9.0, 117.0, 39.0, 0.0, 8....",233,233,233,False,True,False
3,383378,"[100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100...","[9747490, 9749036, 9749025, 9748792, 9748592, ...","[85.0, 18.0, 133.0, 191.0, 331.0, 56.0, 43.0, ...",664,664,664,False,True,False
4,411733,"[20.0, 14.0, 61.0, 55.0, 21.0, 81.0, 100.0, 10...","[9750081, 9750111, 9750039, 9749948, 9749729, ...","[2.0, 4.0, 6.0, 9.0, 1.0, 30.0, 37.0, 5.0, 3.0...",281,281,281,False,True,False


In [298]:
history_df['read_time_fixed_has_nan'].unique()

array([False])

### There both nulls and zeros in the scroll_percentage_fixed column but not in read_time_fixed, i will use the read_time_fixed for interraction matrix or i will have to fix the nans to scroll_percentage_fixed

In [299]:
# # Combine interactions into a single metric (Potential use of scroll_percentage_fixed column)
# def combine_interactions(interactions, interactions2, w1=0.5, w2=0.5):
#     return [w1*i + w2*i2  for i, i2 in zip(interactions, interactions2)]

# history_dfraction_combined'] = history_df(lambda row:   combine_interactions(row['scroll_percentage_fixed'], row['read_time_fixed']) , axis=1)

# history_df ['scroll_percentage_fixed', 'read_time_fixed'], axis=1, inplace=True)

In [300]:
history_df.drop(['scroll_percentage_fixed_len', 'article_id_fixed_len', 'read_time_fixed_len', 'scroll_percentage_fixed_has_0_in', 'read_time_fixed_has_nan' ,'read_time_fixed_has_0_in'], axis=1, inplace=True)

In [301]:
history_df.head(2)

,user_id,scroll_percentage_fixed,article_id_fixed,read_time_fixed
0,750497,"[100.0, 65.0, 100.0, 100.0, 100.0, 100.0, 100....","[9749224, 9749156, 9749224, 9748948, 9748980, ...","[49.0, 5.0, 7.0, 151.0, 214.0, 199.0, 22.0, 64..."
1,22779,"[52.0, 39.0, 62.0, 38.0, 74.0, 19.0, 30.0, 56....","[9749025, 9750090, 9750015, 9750161, 9745750, ...","[4.0, 16.0, 2.0, 9.0, 40.0, 7.0, 9.0, 8.0, 18...."


In [302]:
history_df.shape

(1562, 4)

In [303]:
## Merge behaviours and history-left join
final_df = pd.merge(history_df,behaviors_df,on='user_id',how='outer')
final_df.shape

(1562, 7)

In [304]:
final_df.head()

,user_id,scroll_percentage_fixed,article_id_fixed,read_time_fixed,article_ids_clicked,read_time,scroll_percentage
0,750497,"[100.0, 65.0, 100.0, 100.0, 100.0, 100.0, 100....","[9749224, 9749156, 9749224, 9748948, 9748980, ...","[49.0, 5.0, 7.0, 151.0, 214.0, 199.0, 22.0, 64...","[9789065, 9782845, 9780702, 9788106, 9788921, ...","[34.0, 338.0, 145.0, 119.0, 220.0, 207.0, 251....","[nan, 100.0, 100.0, 100.0, 100.0, nan, nan, 10..."
1,22779,"[52.0, 39.0, 62.0, 38.0, 74.0, 19.0, 30.0, 56....","[9749025, 9750090, 9750015, 9750161, 9745750, ...","[4.0, 16.0, 2.0, 9.0, 40.0, 7.0, 9.0, 8.0, 18....","[9787261, 9784852, 9780702, 9785111, 9782806, ...","[14.0, 58.0, 11.0, 14.0, 10.0, 11.0, 9.0, 61.0...","[nan, 100.0, nan, nan, nan, nan, nan, nan, nan]"
2,373598,"[nan, nan, nan, 59.0, 33.0, 75.0, nan, nan, 76...","[9514481, 9514481, 9111040, 9750389, 9750307, ...","[0.0, 0.0, 0.0, 3.0, 9.0, 117.0, 39.0, 0.0, 8....","[9784559, 9779807, 9783803, 9780439, 9786719, ...","[84.0, 62.0, 235.0, 12.0, 21.0, 50.0, 28.0, 11...","[100.0, 100.0, 100.0, 100.0, nan, nan, nan, 10..."
3,383378,"[100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100...","[9747490, 9749036, 9749025, 9748792, 9748592, ...","[85.0, 18.0, 133.0, 191.0, 331.0, 56.0, 43.0, ...","[9782604, 9782256, 9782808, 9782806, 9790325, ...","[4.0, 94.0, 31.0, 22.0, 122.0, 5.0, 21.0, 21.0...","[100.0, 100.0, nan, 100.0, 100.0, nan, 100.0, ..."
4,411733,"[20.0, 14.0, 61.0, 55.0, 21.0, 81.0, 100.0, 10...","[9750081, 9750111, 9750039, 9749948, 9749729, ...","[2.0, 4.0, 6.0, 9.0, 1.0, 30.0, 37.0, 5.0, 3.0...","[9788907, 9788106, 9783657, 9783752, 9782092, ...","[10.0, 17.0, 12.0, 19.0, 12.0, 15.0, 12.0, 11....","[nan, nan, nan, nan, nan, nan, nan, 100.0, 100..."


In [305]:
def concatenate_lists(df, col1, col2, new_col_name):
    df[new_col_name] = df.apply(lambda row: row[col1] + row[col2] if isinstance(row[col1], list) and isinstance(row[col2], list) else row[col1] if isinstance(row[col1], list) else row[col2] if isinstance(row[col2], list) else [], axis=1)
    return df

In [306]:
final_df = concatenate_lists(final_df,"article_ids_clicked","article_id_fixed","merge_article_ids")
final_df = concatenate_lists(final_df,"read_time","read_time_fixed","merge_read_time")
final_df = concatenate_lists(final_df,"scroll_percentage_fixed",'scroll_percentage',"merge_scroll_percentage")

In [307]:
final_df.shape

(1562, 10)

In [308]:
final_df.head()

,user_id,scroll_percentage_fixed,article_id_fixed,read_time_fixed,article_ids_clicked,read_time,scroll_percentage,merge_article_ids,merge_read_time,merge_scroll_percentage
0,750497,"[100.0, 65.0, 100.0, 100.0, 100.0, 100.0, 100....","[9749224, 9749156, 9749224, 9748948, 9748980, ...","[49.0, 5.0, 7.0, 151.0, 214.0, 199.0, 22.0, 64...","[9789065, 9782845, 9780702, 9788106, 9788921, ...","[34.0, 338.0, 145.0, 119.0, 220.0, 207.0, 251....","[nan, 100.0, 100.0, 100.0, 100.0, nan, nan, 10...","[9789065, 9782845, 9780702, 9788106, 9788921, ...","[34.0, 338.0, 145.0, 119.0, 220.0, 207.0, 251....","[nan, 100.0, 100.0, 100.0, 100.0, nan, nan, 10..."
1,22779,"[52.0, 39.0, 62.0, 38.0, 74.0, 19.0, 30.0, 56....","[9749025, 9750090, 9750015, 9750161, 9745750, ...","[4.0, 16.0, 2.0, 9.0, 40.0, 7.0, 9.0, 8.0, 18....","[9787261, 9784852, 9780702, 9785111, 9782806, ...","[14.0, 58.0, 11.0, 14.0, 10.0, 11.0, 9.0, 61.0...","[nan, 100.0, nan, nan, nan, nan, nan, nan, nan]","[9787261, 9784852, 9780702, 9785111, 9782806, ...","[14.0, 58.0, 11.0, 14.0, 10.0, 11.0, 9.0, 61.0...","[nan, 100.0, nan, nan, nan, nan, nan, nan, nan]"
2,373598,"[nan, nan, nan, 59.0, 33.0, 75.0, nan, nan, 76...","[9514481, 9514481, 9111040, 9750389, 9750307, ...","[0.0, 0.0, 0.0, 3.0, 9.0, 117.0, 39.0, 0.0, 8....","[9784559, 9779807, 9783803, 9780439, 9786719, ...","[84.0, 62.0, 235.0, 12.0, 21.0, 50.0, 28.0, 11...","[100.0, 100.0, 100.0, 100.0, nan, nan, nan, 10...","[9784559, 9779807, 9783803, 9780439, 9786719, ...","[84.0, 62.0, 235.0, 12.0, 21.0, 50.0, 28.0, 11...","[100.0, 100.0, 100.0, 100.0, nan, nan, nan, 10..."
3,383378,"[100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100...","[9747490, 9749036, 9749025, 9748792, 9748592, ...","[85.0, 18.0, 133.0, 191.0, 331.0, 56.0, 43.0, ...","[9782604, 9782256, 9782808, 9782806, 9790325, ...","[4.0, 94.0, 31.0, 22.0, 122.0, 5.0, 21.0, 21.0...","[100.0, 100.0, nan, 100.0, 100.0, nan, 100.0, ...","[9782604, 9782256, 9782808, 9782806, 9790325, ...","[4.0, 94.0, 31.0, 22.0, 122.0, 5.0, 21.0, 21.0...","[100.0, 100.0, nan, 100.0, 100.0, nan, 100.0, ..."
4,411733,"[20.0, 14.0, 61.0, 55.0, 21.0, 81.0, 100.0, 10...","[9750081, 9750111, 9750039, 9749948, 9749729, ...","[2.0, 4.0, 6.0, 9.0, 1.0, 30.0, 37.0, 5.0, 3.0...","[9788907, 9788106, 9783657, 9783752, 9782092, ...","[10.0, 17.0, 12.0, 19.0, 12.0, 15.0, 12.0, 11....","[nan, nan, nan, nan, nan, nan, nan, 100.0, 100...","[9788907, 9788106, 9783657, 9783752, 9782092, ...","[10.0, 17.0, 12.0, 19.0, 12.0, 15.0, 12.0, 11....","[nan, nan, nan, nan, nan, nan, nan, 100.0, 100..."


In [309]:
final_df = final_df[['merge_article_ids','merge_read_time','user_id']]

In [310]:
final_df.head(2)

,merge_article_ids,merge_read_time,user_id
0,"[9789065, 9782845, 9780702, 9788106, 9788921, ...","[34.0, 338.0, 145.0, 119.0, 220.0, 207.0, 251....",750497
1,"[9787261, 9784852, 9780702, 9785111, 9782806, ...","[14.0, 58.0, 11.0, 14.0, 10.0, 11.0, 9.0, 61.0...",22779


In [311]:
# final_df.set_index('user_id',inplace=True)

In [312]:
# final_df = final_df.explode(['merge_scroll_percentage','merge_article_ids']).dropna(subset='merge_scroll_percentage')

In [313]:
# final_df.drop(axis=1,inplace=True,columns='merge_scroll_percentage')

In [314]:
final_df.head()

,merge_article_ids,merge_read_time,user_id
0,"[9789065, 9782845, 9780702, 9788106, 9788921, ...","[34.0, 338.0, 145.0, 119.0, 220.0, 207.0, 251....",750497
1,"[9787261, 9784852, 9780702, 9785111, 9782806, ...","[14.0, 58.0, 11.0, 14.0, 10.0, 11.0, 9.0, 61.0...",22779
2,"[9784559, 9779807, 9783803, 9780439, 9786719, ...","[84.0, 62.0, 235.0, 12.0, 21.0, 50.0, 28.0, 11...",373598
3,"[9782604, 9782256, 9782808, 9782806, 9790325, ...","[4.0, 94.0, 31.0, 22.0, 122.0, 5.0, 21.0, 21.0...",383378
4,"[9788907, 9788106, 9783657, 9783752, 9782092, ...","[10.0, 17.0, 12.0, 19.0, 12.0, 15.0, 12.0, 11....",411733


In [315]:
final_df = final_df.explode('merge_article_ids')

In [316]:
final_df.shape

(25505, 3)

In [317]:
# Creation of interaction matrix

interaction_matrix = pd.DataFrame(columns=final_df.columns)
counter = 0
for user_id in tqdm(np.unique(final_df.index.values).tolist()):
    
    temp_df = final_df[final_df.index == user_id].copy()
    temp_df = temp_df.explode('merge_article_ids')
    temp_df.set_index('user_id',inplace=True)
    temp_lst = []

    # Fix the lists that exist in each cell and keep the values that i care
    temp_counter = 0
    for lst in list(temp_df['merge_read_time']):
        temp_lst.append(lst[temp_counter])
        temp_counter += 1
    
    temp_df['merge_read_time_fixed'] = temp_lst

    # Drop first unneccesary column
    temp_df.drop('merge_read_time', axis=1, inplace= True)
    temp_pivoted_df = temp_df.pivot_table(index = temp_df.index, columns='merge_article_ids', values='merge_read_time_fixed', aggfunc='sum').copy()

    if counter == 0:
        interaction_matrix = temp_pivoted_df.copy()
        #print(interaction_matrix)
    else:
        interaction_matrix = pd.concat([interaction_matrix, temp_pivoted_df], axis=0, ignore_index=False)
    
    counter += 1

C:\Users\giwrg\AppData\Local\Temp\ipykernel_25208\1750402660.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for user_id in tqdm(np.unique(final_df.index.values).tolist()):


  0%|          | 0/1562 [00:00<?, ?it/s]

In [318]:
interaction_matrix.shape

(1562, 1144)

In [319]:
interaction_matrix.head()

merge_article_ids,9777026,9777705,9779365,9780195,9780325,9780482,9780496,9780702,9780769,9780860,9780861,9781476,9781785,9781859,9781875,9781913,9782027,9782407,9782845,9782996,9783137,9783183,9783276,9783757,9783850,9783858,9783965,9784058,9784444,9784662,9785404,9785732,9785799,9785828,9785992,9786288,9786693,9787243,9787332,9787353,9787465,9787524,9787593,9788106,9788125,9788766,9788921,9789065,9789600,9789702,9789721,9789810,9790002,9790121,9790784,9782806,9784044,9784344,9784852,9785111,9786209,9787261,9788666,9142564,9756075,9779007,9779122,9779807,9780439,9781763,9781860,9782423,9783075,9783405,9783803,9784275,9784559,9785017,9786719,9788404,9788950,9789603,9790572,9790752,9790827,9790885,9775978,9780460,9780476,9781086,9781158,9782180,9782256,9782604,9782656,9782808,9782993,9783296,9784298,9785829,9786213,9786222,9786268,9786649,9786932,9788752,9788797,9789642,9789927,9790278,9790325,9790689,9761920,9779248,9779653,9780517,9781354,9781709,9781796,9781838,9781932,9782092,9782438,9782929,9783057,9783657,9783752,9783852,9783952,9784425,9784804,9786563,9787176,9787586,9788907,9789911,9790559,9780384,9781502,9782311,9782421,9783646,9783774,9784160,9784870,9784876,9785790,9786293,9788251,9788547,9788894,9789096,9789279,9790532,9777466,9782722,9785339,9786172,9786567,9052390,9779411,9783024,9784863,9785471,9789494,9777200,9780329,9780874,9780925,9782869,9783379,9783824,9784679,9785835,9787455,9788323,9788898,9789001,9789694,9760046,9778869,9780919,9780968,9781424,9781887,9781991,9782005,9782282,9782391,9782635,9782809,9783754,9784575,9784607,9784710,9785341,9785596,9785899,9786245,9786306,9786529,9786549,9786821,9787510,9787525,9787784,9787855,9788362,9788462,9788664,9788835,9788849,9789037,9789502,9789539,9790066,9790713,6741781,9277736,9770145,9777199,9778795,9780267,9780547,9780677,9782519,9782899,9783019,9783164,9783655,9784642,9786359,9787863,9788188,9788947,9774526,9778796,9779615,9780514,9781013,9781257,9781667,9782126,9782361,9782709,9782915,9784393,9784489,9784696,9784805,9784951,9785019,9785174,9785604,9785665,9785742,9785888,9785892,9786729,9790475,9790574,9790825,9772508,9780283,9782085,9782884,9783237,9785553,9786313,9786566,9787487,9785454,9787553,9787646,9773779,9780960,9789754,9789997,9790025,9769155,9780467,9780983,9781656,9781814,9781856,9782057,9783213,9783404,9784803,9787441,9787722,9789417,9789517,9790756,9771113,9781389,9781558,9782103,9782726,9783043,9784281,9784952,9785016,9785209,9785718,9786243,9786524,9787022,9788151,9788524,9788918,9789471,9790942,9779724,9780210,9780993,9782234,9784875,9786205,9787346,9788225,9789728,9789745,9790360,9782181,9783993,9784856,9785107,9789704,9784591,9786066,9220931,9780909,9781506,9782115,9783643,9785500,9786210,9781636,9785020,9785973,9780561,9782449,9790052,9790476,9778939,9782205,9783349,9783667,9791130,9782108,9782248,9783042,9783314,9783628,9784137,9785487,9785668,9788184,9788466,9789248,9754112,9767830,9772475,9773612,9775621,9780284,9780372,9780697,9780921,9781262,9781316,9781598,9782142,9782418,9782837,9782879,9782919,9783904,9784097,9784406,9784946,9785475,9786906,9787116,9619030,9777846,9779294,9779417,9779657,9779713,9779738,9779777,9779867,9780020,9780300,9780302,9780308,9780348,9780406,9781878,9781906,9786860,9787679,9787931,9788024,9788108,9788692,9789423,9789647,9789676,9789692,9789703,9789710,9790917,9754241,9782074,9782499,9786618,9717946,9780604,9780986,9503296,9726237,9777822,9779181,9779511,9780815,9781535,9783317,9783334,9783626,9783652,9785076,9785260,9785751,9786230,9786495,9787230,9695098,9781467,9788197,9339920,9446756,9768242,9780280,9780498,9781947,9782131,9782290,9784793,9786763,9787767,9788521,9788705,9780374,9785048,9785434,9786351,9790293,9790521,9790645,9219607,9780428,9780580,9781998,9784094,9784214,9788653,9787932,9788239,9788361,9788497,9121218,9654405,9765777,9777858,9782378,9782517,9783732,9786247,9787264,9789379,8860119,9561842,9767490,9770225,9777324,9781057,9782046,9782468,9782763,9782770,9786699,9788145,9788352,9788621,978914

In [320]:
fillna_value = '0'
interaction_matrix.fillna(0, inplace=True)

In [321]:
fillna_value

'0'

In [322]:
# Save the interaction matrix to pickle
interaction_matrix_file_path = f'./files/pickle/interaction_matrix_{type_}_{size}_{fillna_value}.pkl'

interaction_matrix.to_pickle(interaction_matrix_file_path)

In [200]:
# Read interaction matrix pickle file
interaction_matrix_file_path = f'./files/pickle/interaction_matrix_{size}_{fillna_value}.pkl'
interaction_matrix = pd.read_pickle(interaction_matrix_file_path)

FileNotFoundError: [Errno 2] No such file or directory: './files/pickle/interaction_matrix_demo_0.pkl'

## Other

In [ ]:
# # Combine interactions into a single metric
# def combine_interactions(interactions, interactions2, w1=1, w2=1):
#     return [w1*i + w2*i2  for i, i2 in zip(interactions, interactions2)]

# # Flatten the interaction data with combined interactions
# flattened_data = []

# for _, row in history_df.iterrows():
#     user_id = row['user_id']
#     article_ids = row['article_id_fixed']
#     combined_interactions = combine_interactions(row['scroll_percentage_fixed'], row['read_time_fixed'])
    
#     for article_id, interaction_combined in zip(article_ids, combined_interactions):
#         flattened_data.append({
#             'user_id': user_id, 
#             'article_id_fixed': article_id, 
#             'interaction_combined': interaction_combined
#         })

# # Convert to DataFrame
# interaction_df = pd.DataFrame(flattened_data)